In [ ]:
!pip install -r requirements.txt

In [1]:
model = "nova-lite"
tones = ['casual', 'elaborate', 'emojify', 'improve', 'keypoints', 'professional', 'proofread', 'shorten', 'witty']

In [2]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError
import pandas as pd
import os
from importlib import import_module
from tqdm import tqdm

from src.completion import *
from src.format import format_prompt_as_xml
from prompts.prompt_tones import *

In [3]:
from dynaconf import Dynaconf

settings = Dynaconf(settings_files=["settings.toml"], env=model, environments=True)

In [4]:
bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = settings.region)

## Generic

In [5]:
master_sys_prompt = "You are now a writing assistant. You will help users write. Rewrite the following text for the given prompt. Just return the result and do not add comments. Do not assume anything about the user. Do not answer any questions about yourself, only rewrite the question. Do not repeat the input text in the output."

In [6]:
prompts_module = import_module(f"prompts.prompt_tones")

In [7]:
# d = pd.read_csv('~/data/all.csv')

folder_path = os.path.expanduser('~/data')
files = sorted([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
last_file = files[-2]

d = pd.read_csv(os.path.join(folder_path, last_file))

In [8]:
'nova' in settings.model

True

In [9]:
get_completion(settings.model, bedrock_client, 'hey')

"Hey there! How can I help you today? If you have any questions or need assistance with something, feel free to ask and I'll do my best to provide you with helpful information and guidance.\n\nWhether you're looking for advice, information on a particular topic, or just want to chat, I'm here to help. So, what's on your mind?"

In [10]:
for tone in tones:
    print(f'''
    ---------------------
    {tone}
    ---------------------
    ''')
    tone_prompt = getattr(prompts_module, tone.capitalize() + "Prompt")
    queries = d[d['tone'] == tone]['input'].unique()
    p = [format_prompt_as_xml(i, tone_prompt()) for i in tqdm(queries)]
    print(p[0])
    n = len(queries)
    a = batch_get_completions(settings.model, bedrock_client, p, [master_sys_prompt] * n, max_concurrent=n)
    new = pd.DataFrame({
        'input': queries,
        'output': a,
        'tone': tone,
        'model': model
    })
    d = pd.concat([d, new], ignore_index=True)


    ---------------------
    casual
    ---------------------
    


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 6716.67it/s]

  <system>Make this text casual.</system>
  <example>
    <user>My flight lands at 4pm</user>
    <assistant>My flight touches down at 4 o'clock.</assistant>
  </example>
  <user>Good morning, how are you doing today?</user>



    ---------------------
    elaborate
    ---------------------
    


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 98/98 [00:00<00:00, 2543.92it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97/97 [00:00<00:00, 7590.01it/s]

  <system>Make the text as concise as possible without loss of any important information.</system>
  <example>
    <user>Hello, it's been a while. How have you been? We should meet up soon, we have a lot to catch you up on since I got married last year!</user>
    <assistant>Hi, how've you been? Let's catch up soon, lots to share since the wedding!</assistant>
  </example>
  <user>The quick brown fox jumps over the lazy dog. It's a well-known pangram, a sentence that contains all the letters of the alphabet. This phrase has been used to test typewriters, computer keyboards, and other devices that involve text input.</user>



    ---------------------
    witty
    ---------------------
    


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 73/73 [00:00<00:00, 6580.79it/s]

  <system>Make this text witty.</system>
  <example>
    <user>My flight lands at 4pm</user>
    <assistant>My flight touches down at the ungodly hour of 4 pm, just in time to catch the rush hour traffic and arrive home fashionably late.</assistant>
  </example>
  <user>I went to the store to buy some groceries.</user>


In [11]:
d

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,input,output,accuracy,completeness,coherence,conciseness,accuracy_score,completeness_score,coherence_score,conciseness_score,overall_score,tone,model
0,0.0,0.0,0.0,"Good morning, how are you doing today?","Hey there, how's it going?","<reasoning>The output text ""Hey there, how's i...","<reasoning>The output text ""Hey there, how's i...","<reasoning>The Output Text ""Hey there, how's i...","<reasoning>The Output text ""Hey there, how's i...",3.0,2.0,3.0,3.0,2.75,casual,haiku
1,1.0,1.0,1.0,I would like to inquire about the availability...,"Hey, I'm wondering if that product is in stock...",<reasoning>The output text captures the essenc...,<reasoning>The output text fails to cover the ...,<reasoning>The output text maintains a casual ...,"<reasoning>The output text ""Hey, I'm wondering...",2.0,2.0,2.0,1.0,1.75,casual,haiku
2,2.0,2.0,2.0,Could you please provide me with the direction...,"Sure, no problem. Where are you right now? I c...",<reasoning>The output text maintains the casua...,<reasoning>The output text does not provide an...,<reasoning>The output maintains a casual and f...,<reasoning>The output text is concise and does...,3.0,2.0,2.0,3.0,2.50,casual,haiku
3,3.0,3.0,3.0,I am experiencing some difficulties with the s...,I'm having a bit of trouble with the software.,<reasoning>The output text accurately captures...,<reasoning>The output text captures the essenc...,<reasoning>The Output text maintains a casual ...,"<reasoning>The Output text ""I'm having a bit o...",3.0,2.0,3.0,3.0,2.75,casual,haiku
4,4.0,4.0,4.0,Would you be so kind as to pass me the salt?,"Hey, could you hand me the salt?","<reasoning>The output text ""Hey, could you han...",<reasoning>The output text captures the essenc...,"<reasoning>The output text ""Hey, could you han...","<reasoning>The output text ""Hey, could you han...",3.0,2.0,3.0,3.0,2.75,casual,haiku
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,NaN,NaN,NaN,I enjoy going to the beach.,I relish the opportunity to bask in the sun's ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,witty,nova-lite
3806,NaN,NaN,NaN,I need to buy new electronics.,"I must procure some fresh gadgets, lest my cur...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,witty,nova-lite
3807,NaN,NaN,NaN,I enjoy trying new hobbies.,Exploring new hobbies is my favorite way to ke...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,witty,nova-lite
3808,NaN,NaN,NaN,I need to buy new books.,I must replenish my collection of literary tre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,witty,nova-lite


In [13]:
d.groupby(['model', 'tone']).count()

Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  input  output  \
model     tone                                                                  
haiku     casual                  93            93          93     93      93   
          elaborate               99            99          99     99      99   
          emojify                 50            50          50     50      50   
          improve                 97            97          97     97      97   
          keypoints               98            98          98     98      98   
          professional            95            95          95     95      95   
          proofread               91            91          91     91      91   
          shorten                 98            98          98     98      98   
          witty                   91            91          91     91      91   
haiku-3-5 casual                  78             0           0     78      78   
          elaborate               98             0           0     98      98   
          emojify                 50             0           0     50      50   
          improve                 63             0           0     63      63   
          keypoints               97             0           0     97      97   
          professional            68             0           0     68      68   
          proofread               63             0           0     63      63   
          shorten                 97             0           0     97      97   
          witty                   73             0           0     73      73   
nova-lite casual                   0             0           0     78      78   
          elaborate                0             0           0     98      98   
          emojify                  0             0           0     50      50   
          improve                  0             0           0     63      63   
          keypoints                0             0           0     97      97   
          professional             0             0           0     68      68   
          proofread                0             0           0     63      63   
          shorten                  0             0           0     97      97   
          witty                    0             0           0     73      73   
phi       casual                  93            93          93     93      93   
          elaborate               99            99          99     99      99   
          emojify                 50            50          50     50      50   
          improve                 97            97          97     97      97   
          keypoints               98            98          98     98      98   
          professional            95            95          95     95      95   
          proofread               91            91          91     91      91   
          shorten                 98            98          98     98      98   
          witty                   91            91          91     91      91   
qwen      casual                  93            93          93     93      93   
          elaborate               99            99          99     99      99   
          emojify                 50            50          50     50      50   
          improve                 97            97          97     97      97   
          keypoints               98            98          98     98      98   
          professional            95            95          95     95      95   
          proofread               91            91          91     91      91   
          shorten                 98            98          98     98      98   
          witty                   91            91          91     91      91   

                        accuracy  completeness  coherence  conciseness  \
model     tone                                                           
haiku     casual              93            93         93           

In [14]:
t = datetime.now().timestamp()
d.to_csv(f'~/data/all-{t}.csv', index=False)

In [144]:
d = d[~((d['tone'] == tone) * (d['model'] == model))]

In [63]:
d

,Unnamed: 0.1,Unnamed: 0,input,output,accuracy,completeness,coherence,conciseness,accuracy_score,completeness_score,coherence_score,conciseness_score,overall_score,tone,model
0,0,0,"Good morning, how are you doing today?","Hey there, how's it going?","<reasoning>The output text ""Hey there, how's i...","<reasoning>The output text ""Hey there, how's i...","<reasoning>The Output Text ""Hey there, how's i...","<reasoning>The Output text ""Hey there, how's i...",3,2,3,3.0,2.75,casual,haiku
1,1,1,I would like to inquire about the availability...,"Hey, I'm wondering if that product is in stock...",<reasoning>The output text captures the essenc...,<reasoning>The output text fails to cover the ...,<reasoning>The output text maintains a casual ...,"<reasoning>The output text ""Hey, I'm wondering...",2,2,2,1.0,1.75,casual,haiku
2,2,2,Could you please provide me with the direction...,"Sure, no problem. Where are you right now? I c...",<reasoning>The output text maintains the casua...,<reasoning>The output text does not provide an...,<reasoning>The output maintains a casual and f...,<reasoning>The output text is concise and does...,3,2,2,3.0,2.50,casual,haiku
3,3,3,I am experiencing some difficulties with the s...,I'm having a bit of trouble with the software.,<reasoning>The output text accurately captures...,<reasoning>The output text captures the essenc...,<reasoning>The Output text maintains a casual ...,"<reasoning>The Output text ""I'm having a bit o...",3,2,3,3.0,2.75,casual,haiku
4,4,4,Would you be so kind as to pass me the salt?,"Hey, could you hand me the salt?","<reasoning>The output text ""Hey, could you han...",<reasoning>The output text captures the essenc...,"<reasoning>The output text ""Hey, could you han...","<reasoning>The output text ""Hey, could you han...",3,2,3,3.0,2.75,casual,haiku
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2431,2431,86,I had a great time at the party.,"Oh, I'm sure you had a blast at the party, but...",<reasoning>The output text introduces a sarcas...,<reasoning>The output text does not cover any ...,<reasoning>The output text does not maintain c...,<reasoning>The output text fails to maintain c...,1,1,1,1.0,1.00,witty,phi
2432,2432,87,I need to buy new decorations.,"Ah, the quest for the perfect adornments! Let'...",<reasoning>The output text maintains the core ...,<reasoning>The output text successfully captur...,<reasoning>The output text maintains a witty a...,<reasoning>The output text is overly verbose a...,2,3,3,2.0,2.50,witty,phi
2433,2433,88,I enjoy trying new recipes.,"I'm quite the culinary adventurer, always on t...",<reasoning>The output text captures the essenc...,<reasoning>The output text successfully captur...,<reasoning>The output text maintains a coheren...,<reasoning>The output text is overly verbose a...,3,3,3,1.0,2.50,witty,phi
2434,2434,89,I had a busy day at work.,"Oh, the joys of a packed schedule! I suppose i...",<reasoning>The output text does not accurately...,<reasoning>The output text does not cover any ...,<reasoning>The output text maintains a witty a...,<reasoning>The output text does not demonstrat...,2,1,2,1.0,1.50,witty,phi
